In [1]:
import pandas as pd
import openai

In [2]:
%ls

CUI-CUIGPT2.csv                       gptMap-Sapbert.ipynb
CUI-CUILabels.csv                     gptMap-clinicalBert.ipynb
CUI-Phe.csv                           kevin_CUI_PheCode.csv
CUI_Phecode_Map.csv                   phecode_definitions1.2.csv
GPT_init.csv                          phecode_icd9_rolled.csv
Less_0.2_CUI_Phe_Map.csv              phecode_x_map.csv
Less_0.3_CUI_Phe_Map.csv              poorCUI-CUILabels.csv
Mapping_CUI_phecodes_with_string.csv  remap.csv
PheCode_String.csv                    sapbert check.ipynb
README.txt                            sapbert_check.csv
biobert_v1_remap.csv                  sapbert_x_v2_remap.csv
clinicalBert check.ipynb              temp.ipynb
clinicalBert_check.csv                tempRemap.ipynb
clinicalbert_v1_remap.csv             testChatGPT.py
clinicalbert_x_v2_remap.csv           testChatGPT2.py
code-codeLabels.csv                   total_phecode_desc.csv
gptCUI.ipynb


In [3]:
df = pd.read_csv("CUI_Phecode_Map.csv")
df

,cui,cui_string,phecode,phecode_string
0,C0178238,intestinal tract infectious disease nos (disor...,8.0,Intestinal infection
1,C0348099,infection; viral,8.6,Viral Enteritis
2,C0009450,rndx infection unspecified,136.0,Other infectious and parasitic diseases
3,C0013369,infectious diarrheal disease nos,8.0,Intestinal infection
4,C0021342,septic enteritis,8.0,Intestinal infection
...,...,...,...,...
57002,C4076499,dissection of precerebral artery (disorder),442.4,Arterial dissection
57003,C4759468,arthritis of right knee caused by streptococcu...,41.2,Streptococcus infection
57004,C0427512,white blood cell count,288.1,Decreased white blood cell count
57005,C0564296,does not prepare food for eating,389.0,Hearing loss


In [86]:
#GPT SCORE FUNCTIOn
#gpt returns "accuracy score" from 0 - 1

import openai

#api key
openai.api_key = ''

def check_relation(str1, str2):
    messages = [
        {"role": "system", "content": "Please act like a doctor verifying if two medical conditions match. This is for a map, so theoretically the two conditions should ultimately mean the same thing. Respond with ONLY 1 DECIMAL NUMBER on 0 to 1 showing how good the match is. Only 1 NUMBER, NO STRINGS."},
        {"role": "user", "content": f"{str1} & {str2}"},
    ]

    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        max_tokens=3,
        n=1,
        stop=None,
        temperature=0,
    )
    answer = response['choices'][0]['message']['content'].strip().lower()
    return answer


#example usage
str1 = "type 1 diabetes"
str2 = "type 2 diabetes"
answer = check_relation(str1, str2)
#print(f"{answer} {confidence}")
print(answer)





0.9


In [59]:
from tqdm import tqdm

df["answer"] = None  

start_i = 0

with tqdm(total=df.shape[0], desc="Processing") as pbar:
    while start_i < df.shape[0]:
        try:
            str1 = df.iloc[start_i]["cui_string"]
            str2 = df.iloc[start_i]["phecode_string"]
            answer = check_relation(str1, str2)
            df.at[start_i, "answer"] = answer  
            start_i += 1
            pbar.update(1) 
        except Exception as e:
            print(f"Error processing row {start_i}: {e}")
            time.sleep(2)
            print(f"Retrying row {start_i}...")



Processing:   1%|▏                        | 516/57007 [03:26<5:01:35,  3.12it/s]

Error processing row 516: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 22 Jun 2023 18:24:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7db67f1bcf403074-BOS', 'alt-svc': 'h3=":443"; ma=86400'}
Retrying row 516...


Processing:   1%|▎                        | 653/57007 [09:26<5:28:43,  2.86it/s]

Error processing row 653: The server is overloaded or not ready yet.
Retrying row 653...


Processing:   1%|▎                        | 788/57007 [10:49<6:31:08,  2.40it/s]

...

Processing: 100%|█████████████████████▉| 56731/57007 [12:00:12<03:07,  1.47it/s]

Error processing row 56731: The server is overloaded or not ready yet.
Retrying row 56731...


Processing: 100%|█████████████████████▉| 56831/57007 [12:01:51<01:05,  2.70it/s]

Error processing row 56831: The server is overloaded or not ready yet.
Retrying row 56831...


Processing: 100%|██████████████████████| 57007/57007 [12:04:07<00:00,  1.31it/s]


In [61]:
df.to_csv("GPT_init.csv", index = False)

In [90]:
for i in range (df.shape[0]):
    try: 
        df.at[i, "answer"] = float(df.at[i, "answer"])
    except:
        print(df.iloc[i]["answer"])
        df.at[i, "answer"] = -1

i'm sorry
n2b
pt2:


In [92]:
unique_counts = df['answer'].value_counts().sort_index()

In [93]:
unique_counts

answer
-1         3
0.0      438
0.1     1184
0.2     5107
0.3     2678
0.4      107
0.5     1353
0.6     1015
0.7     5018
0.8     9054
0.9    25936
1.0     5114
Name: count, dtype: int64

In [99]:
df2 = df[(df["answer"] <= 0.2) & (df["answer"] >= 0)]
df2.reset_index(drop=True, inplace=True)

In [10]:
Phecode_Str_Map = pd.read_csv("total_phecode_desc.csv")
Phecode_Str_Map.drop("Unnamed: 0", axis = 1, inplace = True)

In [11]:
Phecode_Str_Map

,phecode,phecode_string,category_num,category,sex,icd10_only,phecode_num
0,ID_001,Salmonella,1.0,Infections,Both,0.0,1.0
1,ID_002,Staphylococcus,1.0,Infections,Both,0.0,2.0
2,ID_002.1,Staphylococcus aureus,1.0,Infections,Both,0.0,2.1
3,ID_003,Escherichia coli,1.0,Infections,Both,0.0,3.0
4,ID_004,Streptococcus,1.0,Infections,Both,0.0,4.0
...,...,...,...,...,...,...,...
3846,NaN,Toxic effect of noxious substances eaten as food,NaN,injuries & poisonings,NaN,NaN,988.0
3847,NaN,"Toxic effect of other substances, chiefly nonm...",NaN,injuries & poisonings,NaN,NaN,989.0
3848,NaN,Effects radiation NOS,NaN,injuries & poisonings,NaN,NaN,990.0
3849,NaN,Sepsis and SIRS,NaN,injuries & poisonings,NaN,NaN,994.0


In [13]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# Load sapBert
tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")  
model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")

# Function to calculate embeddings
def calculate_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs[0][0].detach().numpy()
    return embeddings.mean(axis=0)

# Function to find closest Phecode
def find_closest_phecode(embedding):
    distances = Phecode_Str_Map['Embeddings'].apply(lambda x: cosine(x, embedding))
    min_distance = distances.min()
    closest_row = Phecode_Str_Map.iloc[distances.idxmin()]
    closest_phecode = closest_row['phecode_num']
    closest_description = closest_row['phecode_string']
    return closest_phecode, closest_description, min_distance



In [14]:
Phecode_Str_Map['Embeddings'] = Phecode_Str_Map['phecode_string'].apply(calculate_embeddings)


In [15]:
Phecode_Str_Map

,phecode,phecode_string,category_num,category,sex,icd10_only,phecode_num,Embeddings
0,ID_001,Salmonella,1.0,Infections,Both,0.0,1.0,"[0.06343311, -0.3838791, 0.83768797, 0.0182366..."
1,ID_002,Staphylococcus,1.0,Infections,Both,0.0,2.0,"[-0.095842876, -0.4679306, 0.05370157, 0.14271..."
2,ID_002.1,Staphylococcus aureus,1.0,Infections,Both,0.0,2.1,"[0.15883832, -0.6854581, -0.4355055, 0.0953453..."
3,ID_003,Escherichia coli,1.0,Infections,Both,0.0,3.0,"[0.22382578, 0.5253523, 0.28362697, 0.11578793..."
4,ID_004,Streptococcus,1.0,Infections,Both,0.0,4.0,"[-0.39779928, -0.45293403, 0.6459086, 0.405812..."
...,...,...,...,...,...,...,...,...
3846,NaN,Toxic effect of noxious substances eaten as food,NaN,injuries & poisonings,NaN,NaN,988.0,"[-0.5100257, -0.38905105, 0.22832946, 0.124221..."
3847,NaN,"Toxic effect of other substances, chiefly nonm...",NaN,injuries & poisonings,NaN,NaN,989.0,"[-0.6874568, 0.28913742, 0.95847535, -0.261540..."
3848,NaN,Effects radiation NOS,NaN,injuries & poisonings,NaN,NaN,990.0,"[-0.16680177, 0.17387845, 0.65361124, -0.14572..."
3849,NaN,Sepsis and SIRS,NaN,injuries & poisonings,NaN,NaN,994.0,"[-0.72907674, -0.30349538, 0.5481757, 0.178072..."


In [17]:
from tqdm import tqdm

df2['Corrected_Phecode'], df2['Corrected_Description'], df2['Cosine_Similarity'] = zip(*tqdm(df2['cui_string'].apply(lambda x: find_closest_phecode(calculate_embeddings(x))), total=df2.shape[0]))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████| 6729/6729 [00:00<00:00, 937719.17it/s]


In [21]:
df2

,cui,cui_string,phecode,phecode_string,answer,updated_phecode,updated_desc,min_dist
0,C0343400,intestinal; microsporidiosis,136.0,Other infectious and parasitic diseases,0.2,526.3,Intestinal infection due to protozoa,0.359246
1,C0851162,musculoskeletal infective disorder (disorder),591.0,Urinary tract infection,0.1,730.0,Other disorders and symptoms of the musculoske...,0.307413
2,C0411268,intestinal; trichomoniasis,136.0,Other infectious and parasitic diseases,0.2,84.4,Trichomoniasis,0.131256
3,C0036231,sarcosporidiosis (diagnosis),136.0,Other infectious and parasitic diseases,0.2,688.1,Sarcoidosis,0.268182
4,C0035436,active rheumatic fever,480.3,Pneumonia due to fungus (mycoses),0.1,705.1,Rheumatoid arthritis,0.265595
...,...,...,...,...,...,...,...,...
6724,C3532240,hypertrophic mitochondrial cardiomyopathy (dis...,588.0,Disorders resulting from impaired renal function,0.1,414.1,Hypertrophic cardiomyopathy,0.204345
6725,C0563500,does not manage time,389.0,Hearing loss,0.1,325.2,Difficulty walking,0.460526
6726,C0027035,infestation of rectum caused by fly larvae (di...,132.0,"Infestation (lice, mites)",0.2,530.3,Rectal prolapse,0.291468
6727,C0422855,seizures,386.3,Labyrinthitis,0.1,330.3,Convulsions,0.103513


In [20]:
df2.rename(columns={'Corrected_Phecode': 'updated_phecode', 'Corrected_Description': 'updated_desc', 'Cosine_Similarity': 'min_dist'}, inplace=True)


In [22]:
df2.rename(columns={'answer' : 'chatGPT_score'}, inplace = True)

In [23]:
df2.to_csv("sapbert_x_v3_remap.csv")

In [24]:
df2

,cui,cui_string,phecode,phecode_string,chatGPT_score,updated_phecode,updated_desc,min_dist
0,C0343400,intestinal; microsporidiosis,136.0,Other infectious and parasitic diseases,0.2,526.3,Intestinal infection due to protozoa,0.359246
1,C0851162,musculoskeletal infective disorder (disorder),591.0,Urinary tract infection,0.1,730.0,Other disorders and symptoms of the musculoske...,0.307413
2,C0411268,intestinal; trichomoniasis,136.0,Other infectious and parasitic diseases,0.2,84.4,Trichomoniasis,0.131256
3,C0036231,sarcosporidiosis (diagnosis),136.0,Other infectious and parasitic diseases,0.2,688.1,Sarcoidosis,0.268182
4,C0035436,active rheumatic fever,480.3,Pneumonia due to fungus (mycoses),0.1,705.1,Rheumatoid arthritis,0.265595
...,...,...,...,...,...,...,...,...
6724,C3532240,hypertrophic mitochondrial cardiomyopathy (dis...,588.0,Disorders resulting from impaired renal function,0.1,414.1,Hypertrophic cardiomyopathy,0.204345
6725,C0563500,does not manage time,389.0,Hearing loss,0.1,325.2,Difficulty walking,0.460526
6726,C0027035,infestation of rectum caused by fly larvae (di...,132.0,"Infestation (lice, mites)",0.2,530.3,Rectal prolapse,0.291468
6727,C0422855,seizures,386.3,Labyrinthitis,0.1,330.3,Convulsions,0.103513
